In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
'''
import pandas as pd
import json
import os

# 1. 최상위 폴더 경로 설정 (하위 폴더들이 있는 폴더)
folder_path = '/content/drive/MyDrive/NLP project/감정 라벨링데이터'

# 2. 빈 리스트 생성 (여기에 각 파일에서 추출된 데이터를 저장할 것)
all_data = []

# 3. os.walk()를 사용하여 하위 폴더까지 모든 JSON 파일 찾기
for root, dirs, files in os.walk(folder_path):
    for file_name in files:
        if file_name.endswith('.json'):  # JSON 파일만 처리
            file_path = os.path.join(root, file_name)

            # 4. JSON 파일 읽기
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # 5. 필요한 데이터 추출 (utterances에서 listener_empathy가 있는 부분만 추출)
            utterances = data.get('utterances', [])

            for utterance in utterances:
                if utterance.get('listener_empathy') is not None:
                    all_data.append({
                        "text": utterance['text'],
                        "listener_empathy": utterance['listener_empathy']
                    })

# 6. pandas 데이터프레임으로 변환
df = pd.DataFrame(all_data)

# 7. listener_empathy 리스트를 콤마로 연결된 문자열로 변환
df['emotion_label'] = df['listener_empathy'].apply(lambda x: ', '.join(x))

# 8. 최종 데이터프레임 출력 (문장과 감정 라벨)
df_final = df[['text', 'emotion_label']]

# 9. 데이터프레임 확인
df_final.head()

# 데이터프레임 크기 확인 (몇 개의 문장이 처리되었는지)
print(f"총 {len(df_final)}개의 문장이 처리되었습니다.")
'''

In [ ]:
'''
# 데이터프레임 저장
df_final.to_csv('/content/drive/MyDrive/NLP project/final_dataframe.csv', index=False, encoding='utf-8-sig')
'''

In [ ]:
df_final = pd.read_csv('/content/drive/MyDrive/KUBIG CONTEST NLP-Team3/19_최지우/emotion_classifier.csv') #  emotion_classifier csv 파일 불러옴

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/KUBIG CONTEST NLP-Team3/19_최지우/emotion_classifier.csv'

In [ ]:
df_final.head(10)

In [ ]:
df_final.value_counts('emotion_label')

,count
emotion_label,
동조,24513
"위로, 동조",18339
격려,17702
위로,16320
"격려, 동조",14975
조언,12956
"조언, 격려",10120
"조언, 동조",9152
"조언, 위로",8002


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np


In [ ]:

# 가능한 모든 감정 레이블 정의
possible_labels = ["격려", "동조", "위로", "조언"]

#  라벨을 다중 라벨 벡터로 변환
# 예: ["격려", "위로"] -> [1, 0, 1, 0] (격려와 위로는 해당, 동조와 조언은 해당하지 않음)
def encode_labels(labels, possible_labels):
    label_vector = [1 if label in labels else 0 for label in possible_labels]
    return label_vector

df_final['encoded_label'] = df_final['emotion_label'].apply(lambda x: encode_labels(x, possible_labels))

In [ ]:
# 학습/테스트 데이터 분리
train_texts, val_texts, train_labels, val_labels = train_test_split(df_final['text'], df_final['encoded_label'], test_size=0.2, random_state=42)

In [ ]:
'''
# ALBERT 토크나이저 및 모델 로드

from transformers import AlbertTokenizer, AlbertForSequenceClassification, Trainer, TrainingArguments

tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = AlbertForSequenceClassification.from_pretrained('albert-base-v2', num_labels=2)
'''

In [ ]:
''''

# Roberta 토크나이저 모델 및 로드

from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

'''

In [ ]:
#  BERT 토크나이저 및 모델 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(possible_labels))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 6. CUDA 사용 여부 확인 및 모델 이동
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)  # 모델을 GPU로 이동

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
print(device)

cuda


In [ ]:
#  데이터셋 준비
class MultiLabelDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(texts.tolist(), truncation=True, padding=True, max_length=64, return_tensors="pt")
        self.labels = torch.tensor(labels.tolist(), dtype=torch.float32)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)


In [ ]:
train_dataset = MultiLabelDataset(train_texts, train_labels)
val_dataset = MultiLabelDataset(val_texts, val_labels)

In [ ]:
#  학습 설정
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/KUBIG CONTEST NLP-Team3/19_최지우/results',
    num_train_epochs=3,
    save_steps=2_000,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/content/drive/MyDrive/KUBIG CONTEST NLP-Team3/19_최지우/logs',
    logging_steps=1000,
    evaluation_strategy="epoch"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
#  Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [ ]:
#  모델 학습
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.554500,0.528382
2,0.506600,0.502544
3,0.479300,0.494819


TrainOutput(global_step=10908, training_loss=0.5246419875141811, metrics={'train_runtime': 4243.4472, 'train_samples_per_second': 82.237, 'train_steps_per_second': 2.571, 'total_flos': 1.1477406333040128e+16, 'train_loss': 0.5246419875141811, 'epoch': 3.0})

In [ ]:
#  모델 평가
trainer.evaluate()

{'eval_loss': 0.4948192238807678,
 'eval_runtime': 103.3002,
 'eval_samples_per_second': 281.519,
 'eval_steps_per_second': 8.8,
 'epoch': 3.0}

In [ ]:
# 모델과 토크나이저 저장 (경로 따로 설정 필요)
model.save_pretrained('/content/drive/MyDrive/KUBIG CONTEST NLP-Team3/19_최지우/saved_model')
tokenizer.save_pretrained('/content/drive/MyDrive/KUBIG CONTEST NLP-Team3/19_최지우/saved_model')

# 저장된 모델과 토크나이저 불러오기 (경로 따로 설정 필요)
model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/KUBIG CONTEST NLP-Team3/19_최지우/saved_model')
tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/KUBIG CONTEST NLP-Team3/19_최지우/saved_model')

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

# 저장된 모델과 토크나이저 불러오기 (경로 따로 설정 필요)
model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/KUBIG CONTEST NLP-Team3/19_최지우/saved_model')
tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/KUBIG CONTEST NLP-Team3/19_최지우/saved_model')

OSError: Incorrect path_or_model_id: '/content/drive/MyDrive/KUBIG CONTEST NLP-Team3/19_최지우/saved_model'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
def predict_empathy_label(model, tokenizer, text, possible_labels, device='cpu'):
    """
    입력된 문장에 대한 공감 라벨을 예측하는 함수

    Args:
        model: 학습된 BERT 모델 (BertForSequenceClassification)
        tokenizer: BERT 토크나이저 (BertTokenizer)
        text: 예측할 문장 (str)
        possible_labels: 가능한 모든 공감 라벨 리스트 (예: ["격려", "동조", "위로", "조언"])
        device: 사용할 장치 (GPU 또는 CPU)

    Returns:
        predicted_emotions: 예측된 공감 라벨 리스트 (예: ["격려", "위로"])
    """

    # 모델을 평가 모드로 전환
    model.eval()

    # 입력 문장을 BERT 입력 형식으로 변환하고 지정된 장치로 이동
    inputs = tokenizer(text, truncation=True, padding=True, max_length=64, return_tensors="pt").to(device)

    # 모델 예측 수행
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # 시그모이드를 통해 각 감정에 대한 확률로 변환
    predictions = torch.sigmoid(logits)

    # 임계값(threshold)을 설정해 해당 감정에 속하는지 여부를 결정 (0.5 이상일 경우 감정이 해당한다고 판단)
    threshold = 0.5
    predicted_labels = (predictions > threshold).int().tolist()[0]

    # 예측된 라벨을 실제 감정으로 변환
    predicted_emotions = [possible_labels[i] for i, label in enumerate(predicted_labels) if label == 1]
    print(predictions)
    return predicted_emotions


In [ ]:
# 가능한 공감 라벨 정의 (이 예시에서는 ["격려", "동조", "위로", "조언"])
possible_labels = ["격려", "동조", "위로", "조언"]

# 예시 문장
new_sentence = "위로가 나오려면 어떻게 해야 하지?"

# 예측 함수 호출 (학습된 모델, 토크나이저, 예측할 문장, 가능한 공감 라벨, GPU 장치)
predicted_empathy = predict_empathy_label(model, tokenizer, new_sentence, possible_labels, device)

print(f"예측된 공감 라벨: {predicted_empathy}")

tensor([[0.2547, 0.5680, 0.0708, 0.3061]], device='cuda:0')
예측된 공감 라벨: ['동조', '조언']
